In [1]:
import torch
import random
import math

from util import TokenizerUtil

tokenizer = TokenizerUtil()

input_ids, _ = tokenizer.encode('how are you', max_length=6)

input_ids, attention_mask = tokenizer.pad_to_left(input_ids)

input_ids, attention_mask, tokenizer.decode(input_ids)

(tensor([   0,    0,    2, 1139,  708,  692]),
 tensor([0, 0, 1, 1, 1, 1]),
 '<pad><pad><bos>how are you')

In [2]:
import json

with open('dataset/eval.json') as f:
    lines = f.readlines()

lines = random.sample(lines, k=16)
lines = [json.loads(i) for i in lines]

len(lines), lines[0]

(16,
 {'prompt': 'Human: context= CREATE TABLE table_name_62 (roll VARCHAR, decile VARCHAR, name VARCHAR) question= What is the roll number of Te Puke Intermediate, which has a decile of 4? Assistant:',
  'chosen': 'SELECT roll FROM table_name_62 WHERE decile = 4 AND name = "te puke intermediate"',
  'rejected': '',
  'response': 'SELECT roll FROM table_name_62 WHERE decile = 4 AND name = "te puke intermediate"'})

In [3]:
# %run 1.model.ipynb
%run 1.model_gemma2.ipynb

model_actor = torch.load('model/rlhf').to('cuda')

In [4]:
from util import get_generate

for data in lines:
    input_ids, _ = tokenizer.encode(data['prompt'], max_length=128)
    input_ids, attention_mask = tokenizer.pad_to_left(input_ids)

    input_ids = input_ids.unsqueeze(0).to('cuda')
    attention_mask = attention_mask.unsqueeze(0).to('cuda')

    generate = get_generate(model_actor,
                            input_ids,
                            tokenizer.eos_token_id,
                            tokenizer.pad_token_id,
                            max_length=256)

    generate = generate[0, 128:].to('cpu')

    print(tokenizer.decode(generate))
    print(data['chosen'])
    print('===============')

select count(ROLL) from TABLE_NAME_62 where DECILE = 4 and NAME = "4"<eos>
SELECT roll FROM table_name_62 WHERE decile = 4 AND name = "te puke intermediate"
select sum(POINTS) from TABLE_NAME_37 where TEAM_AGAINST = "SOUTH CAROLINA" and TRIES_AGAINST < 8<eos>
SELECT SUM(points_against) FROM table_name_37 WHERE team = "harlequins" AND tries_against < 8
select max(GRID) from TABLE_NAME_80 where TIME = "+44.866" and LAPS > 25<eos>
SELECT MAX(grid) FROM table_name_80 WHERE time = "+44.866" AND laps > 25
select count(OTHERS_PASSENGERS) from TABLE_13836704_6 where AIRPORT = "cHARLES bO"<eos>
SELECT transit_passengers FROM table_13836704_6 WHERE airport = "London Gatwick"
select TITLE from TABLE_NAME_64 where DATE = "2009"<eos>
SELECT title FROM table_name_64 WHERE released = 2009
select min(SCORER_) from TABLE_NAME_91 where REGION__MARKET_ = "31990" and REGION__MARKET_ = "NEW YORK RANG"<eos>
SELECT channels___analog___digital__ FROM table_name_91 WHERE first_air_date = "31 december 1990" AND